## 1) 처리할 법 & LLM 지정
꼭 처리해봐야할 법 조항
| 조항 | 위반 횟수  |
| --- | --- |
| 제29조 | 260 |
| 제26조 | 56 |
| 제34조 | 45 |
| 제21조 | 40 |
| 제24조의2 | 39 |
| 제39조의4 | 35 |
| 제24조 | 34 |
| 전체 데이터 | 397 개 |

In [17]:
test_article = "제15조" 
CODE_GEN_MODEL = "gpt-5"
CHECKLIST_ANSWER_MODEL = "gpt-5-nano"
FALLBACK_JSON_PARSE = "gpt-5-nano"

# 1. 체크리스트 및 코드 생성 

## 1-1) 유틸 함수 선언

In [18]:
# ===========================================
# 🧩 Utility functions for law code generation (clean & minimal)
# ===========================================
from pathlib import Path
import json
import re
from datetime import datetime
from pathlib import Path
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI()

# 파일 읽기 — 담백하게 상대경로 기준
law_path = Path("../../../dataset/PIPA/law/law.json")

with open(law_path, "r", encoding="utf-8") as f:
    data = json.load(f)


# id로 검색 → 해당 조항과 모든 하위 항목을 문자열로 반환
def _extract_article_id(query: str):
    """입력된 문자열에서 '제~조' 형식의 article id 추출"""
    if not isinstance(query, str):
        return None
    match = re.search(r"(제\d+조(?:의\d+)?)", query.strip())
    return match.group(1) if match else None


def format_article_hierarchy(query_id: str):
    """특정 조항(id)과 모든 하위항목을 문자열로 구성"""
    article_id = _extract_article_id(query_id)
    if not article_id:
        return "검색한 문자열에서 '제~조' 형식의 조항 id를 찾을 수 없습니다."

    # 모든 노드를 dict 형태로 구성
    node_by_id = {n["id"]: n for n in data if isinstance(n.get("id"), str)}
    children = {}
    for node in data:
        parent = node.get("parent")
        if isinstance(parent, str):
            children.setdefault(parent, []).append(node["id"])

    # DFS로 하위항목 모두 수집
    def gather(node_id):
        results = []
        for child_id in children.get(node_id, []):
            results.append(child_id)
            results.extend(gather(child_id))
        return results

    if article_id not in node_by_id:
        return f"'{article_id}' 조항을 찾을 수 없습니다."

    target_ids = [article_id] + gather(article_id)
    lines = []
    for tid in target_ids:
        node = node_by_id.get(tid, {})
        var_name = node.get("var_name", "")
        var_part = f" [{var_name}]" if var_name else ""
        content = node.get("content", "")
        lines.append(f"{tid}{var_part} : {content}")
    return "\n".join(lines)


# GPT-5 → JSON 파싱 실패 시 GPT-4o로 재파싱
def parse_with_fallback(output_text: str, prompt: str):
    """GPT-5 결과 JSON 파싱 실패 시 GPT-4o를 이용해 JSON 재생성"""
    try:
        return json.loads(output_text)
    except Exception:
        print("⚠️ GPT-5 결과 JSON 파싱 실패 → GPT-4o로 재시도...")
        messages = [
            {"role": "system", "content": "너는 주어진 문자열에서 JSON만 출력하는 법률 코드화 모델이다. 아래 내용에서 json만 추출하여 반환 반환하시오"},
            {"role": "user", "content": prompt},
        ]
        completion = client.chat.completions.create(
            model=FALLBACK_JSON_PARSE,
            messages=messages,
            response_format={"type": "json_object"},
        )
        return json.loads(completion.choices[0].message.content)

# ===========================================
# ⚙️ pseudocode / variable JSON 저장 함수
# ===========================================
import json
from datetime import datetime
from pathlib import Path

def save_pseudocode_and_variables(law_subset, BASE_VARIABLES, output_dir="outputs"):
    """
    law_subset의 gpt_result에서 id, pseudocode만 추출하여 pseudocode.json으로 저장하고,
    BASE_VARIABLES를 별도 variables.json으로 저장한다.
    """
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    # pseudocode JSON 생성
    pseudocode_data = []
    for node in law_subset:
        try:
            node_id = node.get("id")
            gpt_result = node.get("gpt_result", {})

            if isinstance(gpt_result, dict):
                for key, val in gpt_result.items():
                    if isinstance(val, dict) and "pseudocode" in val:
                        pseudocode_data.append({
                            "id": key,
                            "pseudocode": val["pseudocode"]
                        })
        except Exception as e:
            print(f"⚠️ {node.get('id')} pseudocode 추출 중 오류: {e}")

    # pseudocode 저장
    pseudocode_path = Path(output_dir) / f"{timestamp}.pseudocode.json"
    with open(pseudocode_path, "w", encoding="utf-8") as f:
        json.dump(pseudocode_data, f, ensure_ascii=False, indent=2)

    # variables 저장
    variable_path = Path(output_dir) / f"{timestamp}.variables.json"
    with open(variable_path, "w", encoding="utf-8") as f:
        json.dump(BASE_VARIABLES, f, ensure_ascii=False, indent=2)

    print(f"✅ pseudocode 저장 완료 → {pseudocode_path}")
    print(f"✅ variables 저장 완료 → {variable_path}")


# 자식 수집용 재귀
def gather_children(root_id):
    results = []
    for n in data:
        if n.get("parent") == root_id:
            results.append(n)
            results.extend(gather_children(n["id"]))
    return results

# 제29조와 그 하위 항목들의 리스트 구성
article_id = _extract_article_id(test_article)
node_by_id = {n["id"]: n for n in data if isinstance(n.get("id"), str)}

if article_id in node_by_id:
    root_node = node_by_id[article_id]
    descendants = gather_children(article_id)
    law_subset = [root_node] + descendants
else:
    raise ValueError(f"'{article_id}' 조항을 찾을 수 없습니다.")

print(f"'{article_id}' 및 하위 조항 {len(law_subset)}개 항목 로드 완료.\n")

'제15조' 및 하위 조항 16개 항목 로드 완료.



In [19]:
## 파싱 함수 테스트 
print(format_article_hierarchy("제34조"))

제34조 [LAW_A34] : 
제34조 제1항 [LAW_A34_P1] : 개인정보처리자는 개인정보가 분실ㆍ도난ㆍ유출(이하 이 조에서 “유출등 ”이라 한다)되었음을 알게 되었을 때에는 지체 없이 해당 정보주체에게 다음 각 호의 사항을 알려야 한다. 다만, 정보 주체의 연락처를 알 수 없는 경우 등 정당한 사유가 있는 경우에는 대통령령으로 정하는 바에 따라 통지를 갈음하는 조치를 취할 수 있다.
제34조 제1항 제1호 [LAW_A34_P1_S1] : 유출등이 된 개인정보의 항목
제34조 제1항 제2호 [LAW_A34_P1_S2] : 유출등이 된 시점과 그 경위
제34조 제1항 제3호 [LAW_A34_P1_S3] : 유출등으로 인하여 발생할 수 있는 피해를 최소화하기 위하여 정보주체가 할 수 있는 방법 등에 관한 정보
제34조 제1항 제4호 [LAW_A34_P1_S4] : 개인정보처리자의 대응조치 및 피해 구제절차
제34조 제1항 제5호 [LAW_A34_P1_S5] : 정보주체에게 피해가 발생한 경우 신고 등을 접수할 수 있는 담당부서 및 연락처
제34조 제2항 [LAW_A34_P2] : 개인정보처리자는 개인정보가 유출등이 된 경우 그 피해를 최소화하기 위한 대책을 마련하고 필요한 조치를 하 여야 한다.
제34조 제3항 [LAW_A34_P3] : 개인정보처리자는 개인정보의 유출등이 있음을 알게 되었을 때에는 개인정보의 유형, 유출등의 경로 및 규모 등 을 고려하여 대통령령으로 정하는 바에 따라 제1항 각 호의 사항을 지체 없이 보호위원회 또는 대통령령으로 정하 는 전문기관에 신고하여야 한다. 이 경우 보호위원회 또는 대통령령으로 정하는 전문기관은 피해 확산방지, 피해 복 구 등을 위한 기술을 지원할 수 있다.
제34조 제4항 [LAW_A34_P4] : 제1항에 따른 유출등의 통지 및 제3항에 따른 유출등의 신고의 시기, 방법, 절차 등에 필요한 사항은 대통령령으 로 정한다.


## 1-2) 기본 변수 선언

In [20]:
# ===========================================
# ⚙️ 기본 변수 리스트 초기화 함수
# ===========================================

def init_base_variables():
    """
    개인정보보호법 평가를 위한 기본 비즈니스 변수 목록을 초기화한다.
    (이 함수 호출 시, BASE_VARIABLES를 초기 상태로 되돌림)
    """
    return [
        {
            "variable": "BUSINESS_USES_PERSONAL_INFORMATION",
            "question": "귀사는 고객 또는 이용자의 개인정보를 처리하거나 보유합니까?"
        },
        {
            "variable": "BUSINESS_OUTSOURCES_PROCESSING",
            "question": "귀사는 개인정보 처리 업무를 제3자에게 위탁합니까?"
        }
    ]

## 1-3) 법 -> 코드 함수 선언

In [ ]:
# ===========================================
# ⚙️ 단일 조항을 pseudocode JSON으로 변환하는 함수 (content 미포함 + fallback 최소화)
# ===========================================
import json

def generate_single_article_json(node, full_article, BASE_VARIABLES):
    """
    개별 조항(node)을 GPT-5를 이용해 pseudocode JSON으로 변환.

    Parameters
    ----------
    node : dict
        law_subset의 단일 조항 (id, reference 포함)
    full_article : str
        상위 조항 전체 문자열 (format_article_hierarchy(test_article) 결과)
    BASE_VARIABLES : list[dict]
        전역에서 관리되는 누적 business 변수 리스트

    Returns
    -------
    dict : {
        "id": str,
        "gpt_result": dict  # GPT-5 결과 JSON (fallback 포함)
    }
    """
    test_article = node["id"]
    references = node.get("reference", [])

    # 🔍 reference 중 개인정보보호법만 필터링 후 format_article_hierarchy로 변환
    ref_ids = [r["id"] for r in references if r.get("law") == "개인정보보호법"]
    ref_texts = "\n\n".join(
        [format_article_hierarchy(rid) for rid in ref_ids if isinstance(rid, str)]
    )

    fewshot_examples = """
  제16조 제1항: 개인정보처리자는 제15조 제1항 각 호의 어느 하나에 해당하여 개인정보를 수집하는 경우에는 그 목적에 필요한 최소한의 개인정보를 수집하여야 한다.
  {
    "pseudocode": {
      "condition_pseudocode": "RECIEVE_CONSENT and (LAW_A15_P1_S1['condition'] or LAW_A15_P1_S2['condition'] or LAW_A15_P1_S3['condition'] or LAW_A15_P1_S4['condition'] or LAW_A15_P1_S5['condition'] or LAW_A15_P1_S6['condition'] or LAW_A15_P1_S7['condition'])",
      "legal_pseudocode": "BUSINESS_COLLECTS_MINIMUM_ONLY",
      "action_pseudocode": ""
    },
    "added_variables": [
      {
        "variable": "BUSINESS_COLLECTS_MINIMUM_ONLY",
        "question": "귀사는 고객의 개인정보를 수집할 때 서비스 제공에 반드시 필요한 최소한의 항목만을 수집합니까? 예를 들어 불필요한 생년월일, 주소, 직업, 가족정보 등을 요구하지 않습니까?"
      }
    ]
  }

  제26조 (업무위탁에 따른 개인정보의 처리 제한)
  {
  "pseudocode": {
    "condition_pseudocode": "BUSINESS_OUTSOURCES_PROCESSING",
    "legal_pseudocode": "(LAW_A26_P1['legal'] and LAW_A26_P2['legal'] and LAW_A26_P3['legal'] and LAW_A26_P4['legal'] and LAW_A26_P5['legal'] and LAW_A26_P6['legal'] and LAW_A26_P7['legal'])",
    "action_pseudocode": ""
  },
  "added_variables": [
  ]
  }

  제2조 제1호: 개인정보’란 살아 있는 개인에 관한 정보로서 성명, 주민등록번호 및 영상 등을 통하여 개인을 식별할 수 있는 정보를 말한다.
  {
    "pseudocode": {
      "condition_pseudocode": "BUSINESS_USES_PERSONAL_INFORMATION",
      "legal_pseudocode": "True",
      "action_pseudocode": ""
    },
    "added_variables": []
  }

  {
  "pseudocode": {
    "condition_pseudocode": "USE_WITHOUT_CONSENT and RELATED_TO_ORIGINAL_PURPOSE and SAFETY_MEASURES_TAKEN and NO_DISADVANTAGE_TO_SUBJECT",
    "legal_pseudocode": "True",
    "action_pseudocode": "LAW_A15_P1['legal'] = True"
  },
  "added_variables": [
    {
      "variable": "USE_WITHOUT_CONSENT",
      "question": "귀사는 정보주체의 추가 동의 없이 개인정보를 이용하는 경우가 있습니까?"
    },
    {
      "variable": "RELATED_TO_ORIGINAL_PURPOSE",
      "question": "해당 이용이 개인정보를 처음 수집한 목적과 합리적으로 관련된 범위 내에서 이루어집니까?"
    },
    {
      "variable": "SAFETY_MEASURES_TAKEN",
      "question": "추가 이용 시 암호화 등 안전성 확보에 필요한 조치를 취하고 있습니까?"
    },
    {
      "variable": "NO_DISADVANTAGE_TO_SUBJECT",
      "question": "추가 이용으로 인해 정보주체에게 불이익이 발생하지 않도록 보장하고 있습니까?"
    }
  ]
}

"""


    base_var_str = json.dumps(BASE_VARIABLES, ensure_ascii=False, indent=2)
    prompt = f"""
너는 개인정보보호법 각 조항을 논리적으로 코드화하는 모델이다.
입력된 조항을 분석해 아래 구조의 JSON으로 출력하라.

각 조항은 반드시 아래 필드를 포함한다:
- pseudocode
    - condition_pseudocode: 적용 조건 (다른 조항 condition, business 변수 기반)
    - legal_pseudocode : 위법 여부 판단 논리 (False일 때 위반)
    - action_pseudocode: condition_pseudocode가 참일 때 수행되는 조치
- added_variables: 논리를 위해 꼭 필요한 경우에만 추가되는 새 변수
    - 각 변수는 "variable"과 "question"을 반드시 포함
    - question은 실제 비즈니스가 해당 법을 위반했는지 판단할 수 있는 구체적 질문이어야 하며 절대로 법 조항 문구를 인용하거나 설명해서는 안된다.

규칙:
- 모든 법의 각 장, 절, 조, 항, 호는 두 개의 논리 변수를 갖는다.
   - condition_pseudocode : 해당 법 조항이 적용될 수 있는 조건(True/False)
   - legal_pseudocode : 해당 법 조항이 준수되었을 때 True, 위반 시 False
- 모든 변수명은 다음 규칙을 따른다:
   LAW_[조항식별자]['condition' 또는 'legal']
   예: LAW_A29_P1['condition'], LAW_A29_P1['legal']
- 다른 조항을 참조할 때는 반드시 위 변수명을 사용한다.
- 모든 조항의 legal_pseudocode가 False이면 '위반'으로 간주하고, True이면 '준수'로 간주한다.
- condition_pseudocode가 False이면 그 조항은 평가대상(비적용)에서 제외된다.
- 불가피한 경우에만 added_variables를 추가하되, 그 질문은 반드시 실제 비즈니스 행위에 대한 구체적인 질문이어야 한며 절대로 법 조항 문구를 인용하거나 설명해서는 안된다.
- pseudocode 내 논리 연산은 and, or, not을 사용하며, 다른 조항의 condition/legal 변수를 조합해 표현한다.
- 변수명 예시:
   - LAW_A15_P1 : 개인정보보호법 제15조 제1항
   - LAW_A15_P1_S2 : 개인정보보호법 제15조 제1항 제2호
   - DECREE_A28_P3 : 개인정보보호법 시행령 제28조 제3항
- action_pseudocode는 조항 간 영향을 코드화할 때만 사용한다.
    
다음은 개인정보보호법 조항을 JSON으로 코드화한 예시들이다.
이때, legal_pseudocode가 False일 경우 '위반'으로 간주한다. (True → 준수)

[예제]
{fewshot_examples}

아래는 비즈니스 평가를 위한 기본 변수 목록이다.
이 변수들은 논리 구성에 우선 활용되어야 하며, 불가피한 경우에만 새 변수를 추가하라.

[기본 변수 목록]
{base_var_str}

[전체 조항]
{full_article}

[처리해야하는 조항]
{test_article}

[레퍼런스]
{ref_texts}
"""

    # 🧠 GPT-5 호출
    try:
        response = client.responses.create(
            model=CODE_GEN_MODEL,
            input=prompt,
            reasoning={"effort": "low"},
        )
        output_text = response.output_text
        print(output_text)
        # ⚙️ GPT-5 결과를 직접 JSON으로 파싱 시도
        try:
            result = json.loads(output_text)
        except json.JSONDecodeError:
            # ⚠️ 실패 시 fallback
            print(f"⚠️ {test_article}: GPT-5 결과 JSON 파싱 실패 → fallback 사용")
            result = parse_with_fallback(output_text, prompt)

    except Exception as e:
        print(f"❌ '{test_article}' 처리 중 오류: {e}")
        result = {"id": test_article, "error": str(e)}

    return {
        "id": test_article,
        "gpt_result": result
    }


## 1-4) 메인 루프

In [22]:
# ===========================================
# ⚙️ 메인 루프: 모든 law_subset 조항 반복 처리 (형태 B 전제 + result 직접 저장)
# ===========================================
from tqdm import tqdm
from datetime import datetime
from pathlib import Path
import json

# 1️⃣ BASE_VARIABLES 초기화
BASE_VARIABLES = init_base_variables()
print(f"🔄 BASE_VARIABLES 초기화 완료 ({len(BASE_VARIABLES)}개 기본 변수 로드됨)\n")

# 3️⃣ pseudocode 누적용 리스트
pseudocode_entries = []

# 4️⃣ 메인 반복
for node in tqdm(law_subset, desc="🔄 Generating pseudocode with GPT-5"):
    try:
        # (1) 각 조항의 전체 계층 구조 갱신
        full_article = format_article_hierarchy(node["id"])

        # (2) GPT-5 호출 및 JSON 결과 획득
        result = generate_single_article_json(node, full_article, BASE_VARIABLES)

        # (3) 결과를 law_subset node에 직접 삽입
        node["result"] = result.get("gpt_result", {})
        print(result)
        # (4) pseudocode만 누적 저장
        gpt_json = result.get("gpt_result", {})
        pseudocode = gpt_json.get("pseudocode", {})
        if pseudocode:
            pseudocode_entries.append({
                "id": node["id"],
                "pseudocode": pseudocode
            })

        # (5) added_variables 누적
        added_vars = gpt_json.get("added_variables", [])
        if isinstance(added_vars, list):
            for v in added_vars:
                if isinstance(v, dict) and "variable" in v:
                    if not any(x["variable"] == v["variable"] for x in BASE_VARIABLES):
                        BASE_VARIABLES.append(v)

    except Exception as e:
        print(f"⚠️ {node.get('id')} 처리 중 오류 발생: {e}")

print(f"\n✅ 전체 {len(law_subset)}개 조항 처리 완료!")
print(f"📦 BASE_VARIABLES 누적 개수: {len(BASE_VARIABLES)}")

🔄 BASE_VARIABLES 초기화 완료 (2개 기본 변수 로드됨)



🔄 Generating pseudocode with GPT-5:   6%|▋         | 1/16 [01:23<20:53, 83.56s/it]

[
  {
    "LAW_A15": {
      "pseudocode": {
        "condition_pseudocode": "BUSINESS_USES_PERSONAL_INFORMATION",
        "legal_pseudocode": "(LAW_A15_P1['condition'] and LAW_A15_P1['legal']) and (not LAW_A15_P3['condition'] or LAW_A15_P3['legal']) and (not LAW_A15_P4['condition'] or LAW_A15_P4['legal'])",
        "action_pseudocode": ""
      },
      "added_variables": []
    }
  },
  {
    "LAW_A15_P1": {
      "pseudocode": {
        "condition_pseudocode": "BUSINESS_USES_PERSONAL_INFORMATION",
        "legal_pseudocode": "((LAW_A15_P1_S1['condition'] and LAW_A15_P1_S1['legal']) or (LAW_A15_P1_S2['condition'] and LAW_A15_P1_S2['legal']) or (LAW_A15_P1_S3['condition'] and LAW_A15_P1_S3['legal']) or (LAW_A15_P1_S4['condition'] and LAW_A15_P1_S4['legal']) or (LAW_A15_P1_S5['condition'] and LAW_A15_P1_S5['legal']) or (LAW_A15_P1_S6['condition'] and LAW_A15_P1_S6['legal']) or (LAW_A15_P1_S7['condition'] and LAW_A15_P1_S7['legal'])) and USE_WITHIN_COLLECTION_PURPOSE",
        "action_p

🔄 Generating pseudocode with GPT-5:   6%|▋         | 1/16 [01:31<22:49, 91.27s/it]


KeyboardInterrupt: 

# 2. 체크리스트 대답 & 위법 판독

## 2-1) 위법 데이터 불러오기 (10개만 사용)

In [ ]:
# ===========================================
# ⚙️ Load and preprocess 50 violation cases
# ===========================================
import json
import re

cases_path = "../../../dataset/PIPA/cases/cases.jsonl"

violations = []
with open(cases_path, "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        if i >= 1:  # 최대 10개까지만 읽기
            break
        try:
            data = json.loads(line.strip())

            # --- violated_articles 정제 ---
            raw_articles = data.get("analysis", {}).get("violated_articles", [])
            simplified_articles = []

            for art in raw_articles:
                law_info = art.get("id", {})
                article_id = law_info.get("id", "")
                m = re.search(r"(제\d+조(?:의\d+)?)", article_id)
                if m:
                    simplified_articles.append(m.group(1))

            # 중복 제거
            unique_articles = sorted(list(set(simplified_articles)))

            # 데이터에 반영
            data["analysis"]["violated_articles"] = unique_articles

            violations.append(data)

        except json.JSONDecodeError as e:
            print(f"⚠️ JSON 파싱 오류 (line {i+1}): {e}")

print(f"✅ 총 {len(violations)}개 사례 로드 완료.")
print(f"예시 1개 위반 조항 리스트:\n{violations[0]['analysis']['violated_articles']}")


✅ 총 300개 사례 로드 완료.
예시 1개 위반 조항 리스트:
['제29조']


## 2-2) 유틸 함수

In [ ]:
# ===========================================
# ⚙️ Pseudocode 실행기 (전역 변수 등록 버전)
# ===========================================
import json
from collections import defaultdict

# -------------------------------------------
# 1️⃣ 전역 변수 생성: 법령 조항 + 비즈니스 변수
# -------------------------------------------
def make_runtime_variables(law_subset, BASE_VARIABLES):
    """
    pseudocode 실행을 위해 실제 Python 전역 변수들을 생성한다.
    예: LAW_A29_P1['condition'] 와 BUSINESS_USES_PERSONAL_INFORMATION = True 등
    """
    global_vars = globals()

    # 1. 법령 조항 변수 생성
    for node in law_subset:
        var_name = node.get("var_name")
        if not var_name:
            continue
        if var_name not in global_vars:
            global_vars[var_name] = {"condition": False, "legal": True}

    # 2. 비즈니스 변수 생성
    for v in BASE_VARIABLES:
        var_name = v.get("variable")
        if not var_name:
            continue
        if var_name not in global_vars:
            global_vars[var_name] = False

    print(f"✅ 런타임 변수 등록 완료: {len(law_subset)}개 법령 + {len(BASE_VARIABLES)}개 비즈니스 변수")

# -------------------------------------------
# 2️⃣ condition_pseudocode 평가
# -------------------------------------------
def evaluate_condition(law_id, pseudocode):
    """
    해당 조항의 condition_pseudocode를 실행.
    True이면 condition=True로 업데이트.
    """
    try:
        code = pseudocode.get("condition_pseudocode", "")
        if not code:
            return False
        result = eval(code)
        if "LAW_" in code:  # LAW_xxx['condition'] 참조 가능
            globals()[law_id]["condition"] = bool(result)
        return bool(result)
    except Exception as e:
        print(f"⚠️ {law_id} condition 평가 중 오류: {e}")
        return False

# -------------------------------------------
# 3️⃣ legal_pseudocode 평가
# -------------------------------------------
def evaluate_legal(law_id, pseudocode):
    """
    해당 조항의 legal_pseudocode를 실행.
    True → legal=True, False → legal=False
    """
    try:
        code = pseudocode.get("legal_pseudocode", "")
        if not code:
            return False
        result = eval(code)
        globals()[law_id]["legal"] = bool(result)
        return bool(result)
    except Exception as e:
        print(f"⚠️ {law_id} legal 평가 중 오류: {e}")
        return False

# -------------------------------------------
# 4️⃣ action_pseudocode 실행
# -------------------------------------------
def execute_action(law_id, pseudocode):
    """
    해당 조항의 condition이 True일 때 action_pseudocode를 실행.
    action_pseudocode는 다른 법령 변수를 변경할 수 있음.
    """
    try:
        code = pseudocode.get("action_pseudocode", "")
        if not code:
            return
        # condition이 True일 때만 실행
        if globals()[law_id]["condition"]:
            exec(code, globals())
            print(f"⚙️ {law_id} action_pseudocode 실행 완료")
    except Exception as e:
        print(f"⚠️ {law_id} action 실행 중 오류: {e}")

# ===========================================
# ⚙️ 법령 트리 기반 Pre-order pseudocode 실행기
# ===========================================
from collections import defaultdict

def build_law_tree(law_subset):
    """
    parent 관계를 이용해 트리를 구성한다.
    반환: (children_dict, node_by_id)
    """
    children = defaultdict(list)
    node_by_id = {}
    for node in law_subset:
        node_id = node.get("id")
        parent_id = node.get("parent")
        node_by_id[node_id] = node
        if parent_id:
            children[parent_id].append(node_id)
    return children, node_by_id


def traverse_and_evaluate(root_id, children, node_by_id):
    """
    법령 트리를 pre-order로 순회하며:
      1. evaluate_condition()
      2. (조건이 True면) 하위 항목 재귀 평가
      3. execute_action()
      4. evaluate_legal()
    을 수행한다.
    """
    node = node_by_id[root_id]
    var_name = node.get("var_name")
    pseudocode = node.get("gpt_result", {}).get("pseudocode", {})

    # --- 1️⃣ condition 평가 ---
    cond_result = evaluate_condition(var_name, pseudocode)
    if cond_result:
        globals()[var_name]["condition"] = True

        # --- 2️⃣ 하위 노드 재귀 처리 ---
        for child_id in children.get(root_id, []):
            traverse_and_evaluate(child_id, children, node_by_id)

        # --- 3️⃣ action 실행 ---
        execute_action(var_name, pseudocode)

    # --- 4️⃣ legal 평가 ---
    legal_result = evaluate_legal(var_name, pseudocode)
    globals()[var_name]["legal"] = legal_result

    return legal_result


# ===========================================
# ⚙️ run_preorder_evaluation (test_article 루트 자동 적용)
# ===========================================
def build_tree_by_parent(law_subset):
    """
    parent 필드를 이용해 트리 구조를 만든다.
    반환값:
      - children: {부모_id: [자식_id, ...]}
      - node_by_id: {id: node_dict}
    """
    children = defaultdict(list)
    node_by_id = {}

    for node in law_subset:
        node_id = node.get("id")
        parent_id = node.get("parent")
        if not node_id:
            continue
        node_by_id[node_id] = node
        if parent_id:
            children[parent_id].append(node_id)

    return children, node_by_id


def collect_descendants(test_article, children):
    """
    test_article을 루트로 하여 모든 자손 노드를 재귀적으로 수집한다.
    반환: [test_article 자신 포함 모든 하위 노드 id]
    """
    result = []

    def dfs(current):
        result.append(current)
        for child in children.get(current, []):
            dfs(child)

    dfs(test_article)
    return result


def run_preorder_evaluation(law_subset, test_article):
    """
    test_article (예: '제291조')을 루트로 하여,
    parent 관계를 이용해 트리를 구성하고 pre-order로 pseudocode를 평가한다.
    """
    children, node_by_id = build_tree_by_parent(law_subset)

    # ✅ test_article과 관련된 루트 후보 탐색
    if test_article not in node_by_id:
        candidates = [nid for nid in node_by_id.keys() if nid.startswith(test_article + " ")]
        if candidates:
            root_id = sorted(candidates)[0]  # 첫 번째 항목을 루트로
            print(f"🔍 '{test_article}' 직접 노드 없음 → '{root_id}'를 루트로 설정")
        else:
            print(f"❌ '{test_article}' 관련 노드를 찾을 수 없습니다.")
            return
    else:
        root_id = test_article

    # ✅ root_id 기준으로 모든 하위 노드 수집
    related_ids = collect_descendants(root_id, children)
    print(f"\n🌳 '{test_article}' 트리 구성 완료 — 총 {len(related_ids)}개 노드 평가 시작\n")

    # ✅ pre-order 순회 실행
    for nid in related_ids:
        traverse_and_evaluate(nid, children, node_by_id)

    print(f"\n✅ '{test_article}' 전체 트리 평가 완료\n")

def infer_checklist_answer(business_doc: str, checklist_question: str) -> bool:
    prompt = f"""
너는 개인정보 처리 방식 점검을 위한 자동 판별 모델이다.
아래의 비즈니스 설명을 읽고, 이어지는 질문에 대해 
'예(True)' 또는 '아니오(False)' 중 하나로만 판단하라.

---
[비즈니스 설명]
{business_doc}

[질문]
{checklist_question}

출력은 반드시 다음 중 하나만 포함해야 한다:
- True
- False
다른 설명이나 문장은 포함하지 마라.
"""

    try:
        response = client.responses.create(
            model=CHECKLIST_ANSWER_MODEL,
            input=prompt,
            reasoning={"effort": "low"}
        )
        answer_text = response.output_text.strip()
        # 결과 정규화
        if re.search(r"true", answer_text, re.IGNORECASE):
            return True
        elif re.search(r"false", answer_text, re.IGNORECASE):
            return False
        else:
            print(f"⚠️ 비정형 응답 감지: {answer_text}")
            return False
    except Exception as e:
        print(f"⚠️ 체크리스트 판단 중 오류 발생: {e}")
        return False


## 2-3) 하나의 케이스 위법 판독 함수

In [ ]:
# ===========================================
# ⚖️ 사건별 위법 조항 판별 함수 (문자열형 violated_articles 대응)
# ===========================================
import re

def analyze_case_violation(case, law_subset, BASE_VARIABLES):
    """
    하나의 사건 데이터를 기반으로:
      1. 런타임 변수 등록
      2. 체크리스트 질문에 따라 변수값 확정 (진행률 출력)
      3. 트리 평가 수행
      4. legal=False인 위법 조항의 (조항명, 변수명) 반환

    반환:
      [(조항명, 변수명), ...]  (중복 제거된 리스트)
    """
    # --- 1️⃣ 런타임 변수 등록 ---
    make_runtime_variables(law_subset, BASE_VARIABLES)

    # --- 2️⃣ 체크리스트 변수 확정 ---
    case_content = case["analysis"]["content"]
    total_q = len(BASE_VARIABLES)
    print(f"\n🧾 사건 '{case['analysis']['case_id']}' 체크리스트 평가 시작 ({total_q}개 질문)\n")

    for idx, v in enumerate(BASE_VARIABLES, start=1):
        var_name = v.get("variable")
        question = v.get("question")
        if not var_name or not question:
            continue

        print(f"🧩 [{idx}/{total_q}] {var_name} 질문 확인 중...")

        result = infer_checklist_answer(case_content, question)
        globals()[var_name] = result

        print(f"   └▶ 답변 결과: {result}\n")

    # --- 3️⃣ 트리 평가 수행 ---
    violated_articles = case["analysis"].get("violated_articles", [])
    if not violated_articles:
        print("⚠️ 위반 조항이 없습니다.")
        return []

    # ✅ 변경된 부분: 문자열 리스트 구조 대응
    root_article_id = test_article  # 이전에는 ["id"]["id"]

    run_preorder_evaluation(law_subset, root_article_id)

    # --- 4️⃣ legal=False 인 조항 수집 ---
    violated_nodes = []
    for node in law_subset:
        var_name = node.get("var_name")
        if not var_name or var_name not in globals():
            continue
        try:
            if globals()[var_name]["legal"] is False:
                law_id = node["id"]
                violated_nodes.append((law_id, var_name))
        except Exception:
            continue

    # --- 5️⃣ 조항명/변수명 정규화 ---
    normalized = []
    for law_id, var_name in violated_nodes:
        m = re.search(r"(제\d+조(?:의\d+)?)", law_id)
        simplified_law = m.group(1) if m else law_id

        # "LAW_A15_P1_S1" → "LAW_A15"
        m2 = re.match(r"(LAW_A\d+(?:_\d+)?)", var_name)
        simplified_var = m2.group(1) if m2 else var_name

        normalized.append((simplified_law, simplified_var))

    # --- 6️⃣ 중복 제거 ---
    unique_normalized = list(set(normalized))

    print(f"\n✅ 사건 '{case['analysis']['case_id']}' 처리 완료 — 위법 조항 {len(unique_normalized)}개 검출")
    return unique_normalized


## 2-4) 전체 데이터 성능 확인 함수

In [ ]:
# ===========================================
# ⚖️ 사건별 위법 판독 메인 루프 (결과 반환, 시각화 포함)
# ===========================================
from tqdm import tqdm

def evaluate_violation_detection(violations, law_subset, BASE_VARIABLES, test_article):
    """
    50개 사건을 순차 처리하며 test_article (예: '제29조') 위법 판정 성능을 계산한다.
    실제 violated_articles의 id에 test_article이 포함되어 있는지를 비교하고,
    모델이 동일 test_article에 대해 legal=False로 판단했는지를 대조한다.
    """
    TP = FP = FN = TN = 0
    total_cases = len(violations)
    print(f"\n📊 총 {total_cases}개 사건에 대한 '{test_article}' 위반 판정 시작...\n")

    for idx, case in enumerate(tqdm(violations, desc=f"🔎 Evaluating {test_article}"), start=1):
        case_id = case["analysis"].get("case_id", f"case_{idx}")
        try:
            # 1️⃣ 모델 판단 수행 (분석 실행)
            analyze_case_violation(case, law_subset, BASE_VARIABLES)

            # 2️⃣ 실제 위반 조항들
            actual_articles = case["analysis"].get("violated_articles", [])
            actual_violation = False
            for art in actual_articles:
                if isinstance(art, dict):
                    art_id = art.get("id", {}).get("id", "")
                else:
                    art_id = str(art)
                if test_article in art_id:
                    actual_violation = True
                    break

            # 3️⃣ 모델 예측: test_article 관련 조항이 legal=False 인 경우
            predicted_violation = any(
                (test_article in (node.get("id") or "")) and (node.get("legal") is False)
                for node in law_subset
            )

            # 4️⃣ 혼동행렬 갱신
            if actual_violation and predicted_violation:
                TP += 1
                result = "✅ TP (정확히 위법 판단)"
            elif not actual_violation and predicted_violation:
                FP += 1
                result = "⚠️ FP (잘못된 위법 판단)"
            elif actual_violation and not predicted_violation:
                FN += 1
                result = "⚠️ FN (위법을 놓침)"
            else:
                TN += 1
                result = "✅ TN (정상적으로 준수로 판단)"

            tqdm.write(f"[{idx}/{total_cases}] 사건 {case_id} → {result}")

        except Exception as e:
            tqdm.write(f"⚠️ 사건 {case_id} 처리 중 오류 발생: {e}")

    # --- 5️⃣ 성능 계산 ---
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = (2 * precision * recall / (precision + recall)) if (precision + recall) > 0 else 0
    accuracy = (TP + TN) / total_cases if total_cases > 0 else 0

    return {
        "test_article": test_article,
        "TP": TP, "FP": FP, "FN": FN, "TN": TN,
        "precision": precision, "recall": recall,
        "f1": f1, "accuracy": accuracy
    }


# ===========================================
# 📋 결과 요약 출력 함수
# ===========================================
def print_evaluation_summary(metrics):
    """
    evaluate_violation_detection()의 결과(metrics 딕셔너리)를 표 형태로 출력.
    tabulate 없이 문자열 포맷만 사용.
    """
    test_article = metrics["test_article"]
    print("\n==================== ⚖️ 성능 요약 ====================\n")
    print(f"조항명: {test_article}\n")

    print(f"{'항목':<12}│ {'값':>10}")
    print("-" * 26)
    print(f"{'TP':<12}│ {metrics['TP']:>10}")
    print(f"{'FP':<12}│ {metrics['FP']:>10}")
    print(f"{'FN':<12}│ {metrics['FN']:>10}")
    print(f"{'TN':<12}│ {metrics['TN']:>10}")
    print("-" * 26)
    print(f"{'Precision':<12}│ {metrics['precision']*100:>9.2f}%")
    print(f"{'Recall':<12}│ {metrics['recall']*100:>9.2f}%")
    print(f"{'F1 Score':<12}│ {metrics['f1']*100:>9.2f}%")
    print(f"{'Accuracy':<12}│ {metrics['accuracy']*100:>9.2f}%")
    print("\n======================================================\n")


## 2-5) 50개에 대해 평가 진행

In [ ]:
# 예: 위반 여부 평가
metrics = evaluate_violation_detection(violations, law_subset, BASE_VARIABLES, test_article=test_article)



📊 총 300개 사건에 대한 '제26조' 위반 판정 시작...



🔎 Evaluating 제26조:   0%|          | 0/300 [00:00<?, ?it/s]

✅ 런타임 변수 등록 완료: 12개 법령 + 22개 비즈니스 변수

🧾 사건 '제2021-015-156호' 체크리스트 평가 시작 (22개 질문)

🧩 [1/22] BUSINESS_USES_PERSONAL_INFORMATION 질문 확인 중...
   └▶ 답변 결과: True

🧩 [2/22] BUSINESS_OUTSOURCES_PROCESSING 질문 확인 중...
   └▶ 답변 결과: False

🧩 [3/22] OUTSOURCING_CONTRACT_INCLUDES_PURPOSE_LIMITATION 질문 확인 중...
   └▶ 답변 결과: False

🧩 [4/22] OUTSOURCING_CONTRACT_INCLUDES_SECURITY_MEASURES 질문 확인 중...
   └▶ 답변 결과: False

🧩 [5/22] OUTSOURCING_CONTRACT_INCLUDES_DECREE_SAFE_ITEMS 질문 확인 중...
   └▶ 답변 결과: False

🧩 [6/22] OUTSOURCING_DISCLOSE_TASKS_AND_VENDORS_PUBLIC 질문 확인 중...
   └▶ 답변 결과: False

🧩 [7/22] OUTSOURCING_DISCLOSURE_INCLUDES_SUBPROCESSORS 질문 확인 중...
   └▶ 답변 결과: False

🧩 [8/22] OUTSOURCING_DISCLOSURE_IS_EASILY_ACCESSIBLE 질문 확인 중...
   └▶ 답변 결과: True

🧩 [9/22] OUTSOURCING_DISCLOSURE_IS_UP_TO_DATE 질문 확인 중...
   └▶ 답변 결과: False

🧩 [10/22] OUTSOURCING_INCLUDES_MARKETING_SOLICITATION 질문 확인 중...
   └▶ 답변 결과: False

🧩 [11/22] OUTSOURCING_MARKETING_NOTICE_TO_DATA_SUBJECTS 질문 확인 중...
   └▶ 답변 결과: False

🧩 [1

🔎 Evaluating 제26조:   0%|          | 1/300 [00:37<3:08:27, 37.82s/it]

   └▶ 답변 결과: False


🌳 '제26조' 트리 구성 완료 — 총 12개 노드 평가 시작


✅ '제26조' 전체 트리 평가 완료


✅ 사건 '제2021-015-156호' 처리 완료 — 위법 조항 1개 검출
[1/300] 사건 제2021-015-156호 → ✅ TN (정상적으로 준수로 판단)
✅ 런타임 변수 등록 완료: 12개 법령 + 22개 비즈니스 변수

🧾 사건 '제2021-015-165호' 체크리스트 평가 시작 (22개 질문)

🧩 [1/22] BUSINESS_USES_PERSONAL_INFORMATION 질문 확인 중...
   └▶ 답변 결과: True

🧩 [2/22] BUSINESS_OUTSOURCES_PROCESSING 질문 확인 중...
   └▶ 답변 결과: False

🧩 [3/22] OUTSOURCING_CONTRACT_INCLUDES_PURPOSE_LIMITATION 질문 확인 중...
   └▶ 답변 결과: False

🧩 [4/22] OUTSOURCING_CONTRACT_INCLUDES_SECURITY_MEASURES 질문 확인 중...
   └▶ 답변 결과: False

🧩 [5/22] OUTSOURCING_CONTRACT_INCLUDES_DECREE_SAFE_ITEMS 질문 확인 중...
   └▶ 답변 결과: False

🧩 [6/22] OUTSOURCING_DISCLOSE_TASKS_AND_VENDORS_PUBLIC 질문 확인 중...
   └▶ 답변 결과: False

🧩 [7/22] OUTSOURCING_DISCLOSURE_INCLUDES_SUBPROCESSORS 질문 확인 중...
   └▶ 답변 결과: False

🧩 [8/22] OUTSOURCING_DISCLOSURE_IS_EASILY_ACCESSIBLE 질문 확인 중...
   └▶ 답변 결과: False

🧩 [9/22] OUTSOURCING_DISCLOSURE_IS_UP_TO_DATE 질문 확인 중...
   └▶ 답변 결과: False

🧩 [1

...

   └▶ 답변 결과: False


🌳 '제26조' 트리 구성 완료 — 총 12개 노드 평가 시작


✅ '제26조' 전체 트리 평가 완료


✅ 사건 '제2021-015-174호' 처리 완료 — 위법 조항 1개 검출
[10/300] 사건 제2021-015-174호 → ✅ TN (정상적으로 준수로 판단)
✅ 런타임 변수 등록 완료: 12개 법령 + 22개 비즈니스 변수

🧾 사건 '제2021-015-157호' 체크리스트 평가 시작 (22개 질문)

🧩 [1/22] BUSINESS_USES_PERSONAL_INFORMATION 질문 확인 중...
   └▶ 답변 결과: True

🧩 [2/22] BUSINESS_OUTSOURCES_PROCESSING 질문 확인 중...
   └▶ 답변 결과: False

🧩 [3/22] OUTSOURCING_CONTRACT_INCLUDES_PURPOSE_LIMITATION 질문 확인 중...
   └▶ 답변 결과: False

🧩 [4/22] OUTSOURCING_CONTRACT_INCLUDES_SECURITY_MEASURES 질문 확인 중...
   └▶ 답변 결과: True

🧩 [5/22] OUTSOURCING_CONTRACT_INCLUDES_DECREE_SAFE_ITEMS 질문 확인 중...
   └▶ 답변 결과: False

🧩 [6/22] OUTSOURCING_DISCLOSE_TASKS_AND_VENDORS_PUBLIC 질문 확인 중...
   └▶ 답변 결과: False

🧩 [7/22] OUTSOURCING_DISCLOSURE_INCLUDES_SUBPROCESSORS 질문 확인 중...
   └▶ 답변 결과: False

🧩 [8/22] OUTSOURCING_DISCLOSURE_IS_EASILY_ACCESSIBLE 질문 확인 중...
   └▶ 답변 결과: False

🧩 [9/22] OUTSOURCING_DISCLOSURE_IS_UP_TO_DATE 질문 확인 중...
   └▶ 답변 결과: False

🧩 [1

🔎 Evaluating 제26조:   4%|▎         | 11/300 [07:40<3:26:08, 42.80s/it]  

   └▶ 답변 결과: False


🌳 '제26조' 트리 구성 완료 — 총 12개 노드 평가 시작


✅ '제26조' 전체 트리 평가 완료


✅ 사건 '제2021-015-157호' 처리 완료 — 위법 조항 1개 검출
[11/300] 사건 제2021-015-157호 → ✅ TN (정상적으로 준수로 판단)
✅ 런타임 변수 등록 완료: 12개 법령 + 22개 비즈니스 변수

🧾 사건 '제2021-015-158호' 체크리스트 평가 시작 (22개 질문)

🧩 [1/22] BUSINESS_USES_PERSONAL_INFORMATION 질문 확인 중...
   └▶ 답변 결과: True

🧩 [2/22] BUSINESS_OUTSOURCES_PROCESSING 질문 확인 중...
   └▶ 답변 결과: False

🧩 [3/22] OUTSOURCING_CONTRACT_INCLUDES_PURPOSE_LIMITATION 질문 확인 중...
   └▶ 답변 결과: False

🧩 [4/22] OUTSOURCING_CONTRACT_INCLUDES_SECURITY_MEASURES 질문 확인 중...
   └▶ 답변 결과: False

🧩 [5/22] OUTSOURCING_CONTRACT_INCLUDES_DECREE_SAFE_ITEMS 질문 확인 중...
   └▶ 답변 결과: False

🧩 [6/22] OUTSOURCING_DISCLOSE_TASKS_AND_VENDORS_PUBLIC 질문 확인 중...
   └▶ 답변 결과: False

🧩 [7/22] OUTSOURCING_DISCLOSURE_INCLUDES_SUBPROCESSORS 질문 확인 중...
   └▶ 답변 결과: False

🧩 [8/22] OUTSOURCING_DISCLOSURE_IS_EASILY_ACCESSIBLE 질문 확인 중...
   └▶ 답변 결과: False

🧩 [9/22] OUTSOURCING_DISCLOSURE_IS_UP_TO_DATE 질문 확인 중...
   └▶ 답변 결과: False

🧩 [

🔎 Evaluating 제26조:   4%|▎         | 11/300 [08:08<3:33:54, 44.41s/it]


KeyboardInterrupt: 

## 2-6) 결과 정리 

In [ ]:
print_evaluation_summary(metrics)


==================== ⚖️ 성능 요약 ====================

조항명: 제26조

항목          │          값
--------------------------
TP          │          0
FP          │          0
FN          │         16
TN          │        284
--------------------------
Precision   │      0.00%
Recall      │      0.00%
F1 Score    │      0.00%
Accuracy    │     94.67%




## 3) 저장

In [ ]:
# ===========================================
# 💾 Save outputs (with legal=False marking)
# ===========================================
import os
import json
import re
from pathlib import Path
from datetime import datetime
try:
    from zoneinfo import ZoneInfo
except Exception:
    ZoneInfo = None  # fallback

# ───────────────────────────────────────────
# 🧱 0. 기본 경로 및 타임스탬프 폴더 생성
# ───────────────────────────────────────────
base_output = Path("outputs")
base_output.mkdir(exist_ok=True)

now = datetime.now(ZoneInfo("Asia/Seoul")) if ZoneInfo else datetime.now()
timestamp = now.strftime("%Y%m%d_%H%M%S")

output_dir = base_output / timestamp
output_dir.mkdir(exist_ok=True)
print(f"📁 Created output directory: {output_dir}\n")

# ───────────────────────────────────────────
# 🔹 1. pseudocode.json 저장
# ───────────────────────────────────────────
pseudocode_entries = []
for node in law_subset:
    node_id = node.get("id")
    result = node.get("result") or node.get("gpt_result") or {}
    pseudo = None

    if isinstance(result, dict):
        if "pseudocode" in result:
            pseudo = result["pseudocode"]
        else:
            for k, v in result.items():
                if isinstance(v, dict) and "pseudocode" in v:
                    pseudo = v["pseudocode"]
                    break

    if pseudo:
        pseudocode_entries.append({
            "id": node_id,
            "var_name": node.get("var_name"),
            "pseudocode": pseudo
        })

pseudocode_path = output_dir / f"{timestamp}.pseudocode.json"
with open(pseudocode_path, "w", encoding="utf-8") as fp:
    json.dump(pseudocode_entries, fp, ensure_ascii=False, indent=2)

print(f"💾 Saved pseudocode JSON: {pseudocode_path} ({len(pseudocode_entries)} entries)")

# ───────────────────────────────────────────
# 🔹 2. variables.json 저장
# ───────────────────────────────────────────
variables_path = output_dir / f"{timestamp}.variables.json"
with open(variables_path, "w", encoding="utf-8") as fp:
    json.dump(BASE_VARIABLES, fp, ensure_ascii=False, indent=2)

print(f"💾 Saved variable JSON: {variables_path} ({len(BASE_VARIABLES)} variables)\n")

# ───────────────────────────────────────────
# 🔹 3. 위법 케이스별 텍스트 저장
# ───────────────────────────────────────────
def safe_filename(s):
    s = re.sub(r"[\\/*?\"<>|:]+", "_", s)
    s = re.sub(r"\s+", "_", s)
    return s[:160]

for i, case in enumerate(violations, start=1):
    case_analysis = case.get("analysis", {})
    case_id = case_analysis.get("case_id", f"case_{i}")
    source_path = case.get("source_path", "")
    content = case_analysis.get("content", "")
    violated_articles = case_analysis.get("violated_articles", [])

    fname = safe_filename(f"{i}_{case_id}.txt")
    txt_path = output_dir / fname

    # 🧩 legal=False 조항 찾기
    illegal_nodes = []
    for node in law_subset:
        if node.get("legal") is False:
            illegal_nodes.append(f"- {node.get('id', '')} [{node.get('var_name', '')}]")

    with open(txt_path, "w", encoding="utf-8") as f:
        f.write(f"Case Index: {i}\n")
        f.write(f"Case ID: {case_id}\n")
        f.write(f"Source Path: {source_path}\n")
        f.write("-" * 60 + "\n\n")

        f.write("== Business Content ==\n")
        f.write(content + "\n\n")

        f.write("== Violated Articles (from case file) ==\n")
        if isinstance(violated_articles, list):
            for art in violated_articles:
                f.write(f"- {art}\n")
        else:
            f.write(f"- {str(violated_articles)}\n")
        f.write("\n")

        f.write("== Checklist Questions & Answers ==\n")
        for j, v in enumerate(BASE_VARIABLES, start=1):
            var = v.get("variable", f"VAR_{j}")
            q = v.get("question", "")
            try:
                ans = infer_checklist_answer(content, q)
            except Exception as e:
                ans = False
                print(f"⚠️ {case_id} / {var} 질문 중 오류: {e}")
            f.write(f"[{j}] {var}\nQ: {q}\nA: {ans}\n\n")

        f.write("== Violated by Model (legal=False) ==\n")
        if illegal_nodes:
            for line in illegal_nodes:
                f.write(line + "\n")
        else:
            f.write("(None)\n")

    print(f"[{i}/{len(violations)}] Saved: {txt_path}")

print(f"\n✅ All outputs saved under: {output_dir}")


📁 Created output directory: outputs/20251103_134005

💾 Saved pseudocode JSON: outputs/20251103_134005/20251103_134005.pseudocode.json (11 entries)
💾 Saved variable JSON: outputs/20251103_134005/20251103_134005.variables.json (22 variables)

[1/300] Saved: outputs/20251103_134005/1_제2021-015-156호.txt
[2/300] Saved: outputs/20251103_134005/2_제2021-015-165호.txt
[3/300] Saved: outputs/20251103_134005/3_제2021-015-166호.txt
[4/300] Saved: outputs/20251103_134005/4_제2021-015-167호.txt
[5/300] Saved: outputs/20251103_134005/5_제2021-015-168호.txt
[6/300] Saved: outputs/20251103_134005/6_제2021-015-169호.txt
[7/300] Saved: outputs/20251103_134005/7_제2021-015-170호.txt
[8/300] Saved: outputs/20251103_134005/8_제2021-015-172호.txt
[9/300] Saved: outputs/20251103_134005/9_제2021-015-173호.txt
[10/300] Saved: outputs/20251103_134005/10_제2021-015-174호.txt
[11/300] Saved: outputs/20251103_134005/11_제2021-015-157호.txt
[12/300] Saved: outputs/20251103_134005/12_제2021-015-158호.txt
[13/300] Saved: outputs/20251103_

KeyboardInterrupt: 